## Guitar Chord Classifier
*Brian Cassriel & Zack Rosenthal*

This project implements a Convolutional Neural Network (CNN) to classify guitar chords from audio recordings. The model is trained on spectrograms generated from waveform audio files of various guitar chords.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

The dataset that we are using is the IDMT-SMT-GUITAR_V2 dataset, specifically "dataset4", which contains recordings of songs played on a guitar and annotations about chords played at different timestamps.

The dataset is not useful to us in its current state, so we will use its data to create a new dataset with only single chord waveforms in folders with chord labels.

In [ ]:
dataset4_path = './IDMT-SMT-GUITAR_V2/dataset4'

chord_label_dataset_path = "./chord_clips"
os.makedirs(chord_label_dataset_path, exist_ok=True)

print("Source root:", dataset4_path)
print("Destination root:", chord_label_dataset_path)

Source root: ./IDMT-SMT-GUITAR_V2/dataset4
Destination root: ./chord_clips


The following set of functions create catgeorized clips of chords as new files from full-length audio files with annotation.

In [ ]:
import soundfile as sf
import librosa
import pandas as pd

def populate_chord_clips(dataset_path):
    clip_count = 0

    for dirpath, dirnames, filenames in os.walk(dataset_path):
        if os.path.basename(dirpath).lower() != "audio":
            continue
        print("Scanning:", dirpath)
        for fname in filenames:
            if not fname.lower().endswith(".wav"):
                continue
            annotation_fname = fname[:-4] + ".csv"
            annotation_path = os.path.join(os.path.dirname(dirpath), "annotation/chords", annotation_fname)
            audio_path = os.path.join(dirpath, fname)
            clips, sr = get_chord_clips_from_wav(audio_path, annotation_path)
            print(f"  Found {len(clips)} chord clips in {fname}")
            save_clips_as_single_files(clips, fname, chord_label_dataset_path, sr)
            clip_count += len(clips)
    print("Created", clip_count, "chord clips.")

def get_chord_clips_from_wav(wav_filename: str, annotation_filename: str) -> str | None:
    audio, sr = librosa.load(wav_filename)
    clips = []
    annotation_csv = pd.read_csv(annotation_filename)
    for (index, row) in annotation_csv.iterrows():
        chord_label = get_chord_label(row)
        if chord_label is None:
            continue
        start_sample, end_sample = get_sample_bounds(annotation_csv, index, row, chord_label, audio, sr)
        clip_audio = audio[start_sample:end_sample]
        clips.append((clip_audio, chord_label))
    return clips, sr

def save_clips_as_single_files(clips, original_fname, destination_root, sr):
    for idx, (clip_audio, chord_label) in enumerate(clips):
        dst_dir = os.path.join(destination_root, chord_label)
        os.makedirs(dst_dir, exist_ok=True)

        base_name = original_fname[:-4]
        dst_filename = f"{base_name}_clip{idx}.wav"
        dst_path = os.path.join(dst_dir, dst_filename)
        sf.write(dst_path, clip_audio, samplerate=sr)

def get_chord_label(row):
    # The chord label is always the first character after the colon. If the chord is flat, we include that as well.
    chord_descriptor = row.iloc[1]
    if len(row) < 2 or ':' not in str(chord_descriptor):
        return None
    chord_name = chord_descriptor.split(":")[1]
    chord_label = chord_name[0]
    if (chord_label == 'N'):
        return None
    if (len(chord_name) > 1 and chord_name[1] == 'b'):
        chord_label = chord_label + 'b'
    return chord_label

def get_sample_bounds(annotation_csv, index, row, chord_label, audio, sr):
    time_sec = row.iloc[0]
    start_sample = int(time_sec * sr)
    end_sample = get_end_sample(annotation_csv, index, chord_label, audio, sr)
    return start_sample, end_sample

def get_end_sample(annotation_csv, index, chord_label, audio, sr):
    if (index + 1) < len(annotation_csv):
        # Look ahead to find the next chord label. That will be the end time for this chord.
        next_index = index + 1
        current_chord = chord_label
        while next_index < len(annotation_csv):
            next_row = annotation_csv.iloc[next_index]
            if len(next_row) < 2 or ':' not in str(next_row.iloc[1]):
                next_index += 1
                continue
            next_chord = next_row.iloc[1].split(":")[1].split("/")[0]
            if next_chord != current_chord:
                end_sample = int(next_row.iloc[0] * sr)
                break
            next_index += 1
        else:
            # No different chord found, use end of audio.
            end_sample = len(audio)
    else:
        end_sample = len(audio)
    return end_sample

In [ ]:
populate_chord_clips(dataset4_path)

Scanning: ./IDMT-SMT-GUITAR_V2/dataset4/acoustic_mic/slow/pop/audio
  Found 8 chord clips in pop_1_130BPM.wav
  Found 15 chord clips in pop_7_60BPM.wav
  Found 11 chord clips in pop_4_100BPM.wav
  Found 7 chord clips in pop_3_130BPM.wav
  Found 15 chord clips in pop_6_90BPM.wav
  Found 11 chord clips in pop_8_85BPM.wav
  Found 9 chord clips in pop_5_78BPM.wav
  Found 8 chord clips in pop_2_120BPM.wav
Scanning: ./IDMT-SMT-GUITAR_V2/dataset4/acoustic_mic/slow/rock_blues/audio
  Found 15 chord clips in rock_2_100BPM.wav
  Found 95 chord clips in rock_7_100BPM.wav
  Found 95 chord clips in rock_8_100BPM.wav
  Found 15 chord clips in rock_5_100BPM.wav
  Found 10 chord clips in rock_6_100BPM.wav
  Found 11 chord clips in rock_3_100BPM.wav
  Found 7 chord clips in rock_4_110BPM.wav
  Found 7 chord clips in rock_1_100BPM.wav
Scanning: ./IDMT-SMT-GUITAR_V2/dataset4/acoustic_mic/slow/metal/audio
  Found 6 chord clips in metal_3_100BPM.wav
  Found 5 chord clips in metal_6_110BPM.wav
  Found 5 cho

In [ ]:
train_dataset, test_dataset = tf.keras.utils.audio_dataset_from_directory(
    directory=chord_label_dataset_path,
    batch_size=None,
    validation_split=0.2,
    seed=0,
    output_sequence_length=16000,
    subset='both'
)
print("Chord classes:", train_dataset.class_names)

Found 2158 files belonging to 12 classes.
Using 1727 files for training.
Using 431 files for validation.
Chord classes: ['A', 'Ab', 'B', 'Bb', 'C', 'D', 'Db', 'E', 'Eb', 'F', 'G', 'Gb']


The following code snippit defines a function to convert waveform audio to a spectrogram. The spectrograms are then used in the CNN.

In [ ]:
spectrogram_shape = (128, 128, 1)

def get_spectrogram(waveform):
    waveform = tf.squeeze(waveform, axis=-1)
    spectrogram = tf.signal.stft(
        waveform,
        frame_length=255,
        frame_step=128
    )
    spectrogram = tf.abs(spectrogram)
    spectrogram = spectrogram[..., tf.newaxis]  # add channel dim
    spectrogram = tf.image.resize(spectrogram, [spectrogram_shape[0], spectrogram_shape[1]])

    return spectrogram


In [ ]:
def make_spec_ds(ds, batch_size=64):
    ds = ds.map(
        lambda audio, label: (get_spectrogram(audio), label),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

train_spectrogram_ds = make_spec_ds(train_dataset)
test_spectrogram_ds = make_spec_ds(test_dataset)

for specs, labels in train_spectrogram_ds.take(1):
    print("Spectrogram batch shape:", specs.shape)
    print("Labels batch shape:", labels.shape)
    break


Spectrogram batch shape: (64, 128, 128, 1)
Labels batch shape: (64,)


The code below takes a train sample to display to validate functionality

In [ ]:
import matplotlib.pyplot as plt

# Take one sample from the train_spectrogram_ds to display
for specs, labels in train_spectrogram_ds.take(1):
    break

# Get the first spectrogram and its label from the batch
sample_spectrogram = specs[0].numpy()
sample_label_idx = labels[0].numpy()
sample_label = train_dataset.class_names[sample_label_idx]

plt.figure(figsize=(10, 6))
plt.imshow(tf.transpose(sample_spectrogram)[0], origin='lower', aspect='auto', cmap='magma')
plt.colorbar(format='%+2.0f dB')
plt.title(f'Sample Spectrogram of a {sample_label} chord')
plt.xlabel('Time')
plt.ylabel('Frequency')
plt.show()

Convolutional Neural Network that takes in spectrograms of guitar chords and classifies them into chord categories.

In [ ]:
num_classes = len(train_dataset.class_names)

model = Sequential([
    tf.keras.layers.Input(shape=spectrogram_shape),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 12)             │         1,548 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,612 (12.61 MB)

 Trainable params: 3,305,612 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
EPOCHS = 20
model.fit(
    train_spectrogram_ds,
    validation_data=test_spectrogram_ds,
    epochs=EPOCHS
)

Epoch 1/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 328ms/step - accuracy: 0.7771 - loss: 0.6893 - val_accuracy: 0.5870 - val_loss: 1.4245
Epoch 2/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 333ms/step - accuracy: 0.8466 - loss: 0.5071 - val_accuracy: 0.6079 - val_loss: 1.5263
Epoch 3/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 319ms/step - accuracy: 0.8761 - loss: 0.3816 - val_accuracy: 0.6148 - val_loss: 1.5678
Epoch 4/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 321ms/step - accuracy: 0.9155 - loss: 0.2713 - val_accuracy: 0.6241 - val_loss: 1.6867
Epoch 5/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 330ms/step - accuracy: 0.9207 - loss: 0.2632 - val_accuracy: 0.6195 - val_loss: 1.8359
Epoch 6/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 345ms/step - accuracy: 0.9531 - loss: 0.1748 - val_accuracy: 0.6334 - val_loss: 1.8735
Epoch 7/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 323ms/step - accuracy: 0.9635 - loss: 0.1212 - val_accuracy: 0.6381 - val_loss: 1.9653
Epoch 8/20
27/27 ━━━━━━━━━━━━━━━━━━━━ 9s 328ms/step - accuracy: 0.9809 - loss: 0.0777 - val_accuracy: 0.

In [ ]:
def load_and_preprocess(filepath):
    audio_binary = tf.io.read_file(filepath)
    audio, sr = tf.audio.decode_wav(audio_binary)
    audio = tf.squeeze(audio, axis=-1)
    audio = audio[:16000] # Pad/trim to 16000 samples (1 sec)
    audio = tf.pad(audio, [[0, max(0, 16000 - tf.shape(audio)[0])]])
    audio = tf.expand_dims(audio, axis=-1)
    spec = get_spectrogram(audio)
    spec = tf.expand_dims(spec, axis=0)
    return spec

def predict_chord(filepath):
    spec = load_and_preprocess(filepath)
    pred = model.predict(spec)
    idx = np.argmax(pred)
    confidence = float(np.max(pred))

    class_names = train_dataset.class_names
    print("File:", filepath)
    print("Predicted label:", class_names[idx])
    print("Confidence:", confidence)

In [ ]:
for dirpath, dirnames, filenames in os.walk("./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio"):
    for filename in filenames:
        if filename.endswith(".wav"):
            filepath = os.path.join(dirpath, filename)
            predict_chord(filepath)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio/2-E1-Minor 02.wav
Predicted label: E
Confidence: 0.4165397882461548
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio/2-E1-Minor 03.wav
Predicted label: E
Confidence: 0.42304912209510803
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio/2-E1-Minor 01.wav
Predicted label: E
Confidence: 0.4222051203250885
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio/2-E1-Minor 00.wav
Predicted label: E
Confidence: 0.3974781930446625
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Clean Neck SC Chords/audio/2-E1-Minor 04.wav
Predicted label: E
Confidence: 0.4462108016014099
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
File: ./IDMT-SMT-GUITAR_V2/dataset1/Fender Strat Cl